In [ ]:
#import library
%pip install py_stringmatching

In [2]:
import py_stringmatching as sm
import pandas as pd

In [2]:
hospital_accounts = pd.read_csv('../data/hospital_account_info.csv')
hospital_reimbursement = pd.read_csv('../data/hospital_reimbursement.csv')

In [4]:
accountNameList = hospital_accounts['Facility Name'].tolist()
reimbursementNameList = hospital_reimbursement['Provider Name'].tolist()
accountNameTokens = []
reimbursementNameTokens = []
ws_tok = sm.WhitespaceTokenizer()
for name in accountNameList:
    accountNameTokens.append(ws_tok.tokenize(name))
for name in reimbursementNameList:
    reimbursementNameTokens.append(ws_tok.tokenize(name))
nameTokens = accountNameTokens + reimbursementNameTokens

In [ ]:
soft_tfidf = sm.SoftTfIdf(nameTokens, sim_func=sm.Jaro().get_raw_score, threshold=0.96)
rows = []
for i in range (0, len(accountNameTokens)):
    for j in range(0, len(reimbursementNameTokens)):
        rows.append([hospital_accounts.at[i,'Account_Num'], hospital_reimbursement.at[j, 'Provider_Num'], 
        soft_tfidf.get_raw_score(accountNameTokens[i], reimbursementNameTokens[j]), hospital_accounts.at[i,'Facility Name'], hospital_reimbursement.at[j, 'Provider Name'],
        hospital_accounts.at[i,'Address'], hospital_reimbursement.at[j, 'Provider Street Address']])

In [7]:
joinTable = pd.DataFrame(rows, columns =['Account_Num', 'Provider_Num', 'Similar Score', 'Facility Name', 'Provider Name', 'Address', 'Provider Street Address'])
#Sắp xếp điểm giảm dần
joinTable.sort_values(by='Similar Score', ascending=False, inplace=True)
joinTable.head(50)

,Account_Num,Provider_Num,Similar Score,Facility Name,Provider Name,Address,Provider Street Address
164600,55272,556917,1.0,SCOTTSDALE OSBORN MEDICAL CENTER,SCOTTSDALE OSBORN MEDICAL CENTER,7400 EAST OSBORN ROAD,7400 EAST OSBORN ROAD
2927349,87831,122546,1.0,CENTRAL MAINE MEDICAL CENTER,CENTRAL MAINE MEDICAL CENTER,300 MAIN STREET,300 MAIN STREET
10001375,36698,633926,1.0,FRANCISCAN HEALTH CROWN POINT,FRANCISCAN HEALTH CROWN POINT,1201 S MAIN ST,1201 S MAIN ST
13765590,79158,140886,1.0,BANNER HEART HOSPITAL,BANNER HEART HOSPITAL,6750 EAST BAYWOOD AVENUE,6750 EAST BAYWOOD AVENUE
6665716,63329,218382,1.0,ST LUKES HOSPITAL,ST LUKES HOSPITAL,1026 A AVE NE,232 S WOODS MILL RD
12167146,12559,736338,1.0,ST ELIZABETH FT THOMAS,ST ELIZABETH FT THOMAS,85 NORTH GRAND AVENUE,85 NORTH GRAND AVENUE
2580549,15932,702922,1.0,METHODIST HEALTHCARE MEMPHIS HOSPITALS,METHODIST HEALTHCARE MEMPHIS HOSPITALS,1265 UNION AVE SUITE 700,1265 UNION AVE SUITE 700
9746745,47056,605720,1.0,INOVA ALEXANDRIA HOSPITAL,INOVA ALEXANDRIA HOSPITAL,4320 SEMINARY RD,4320 SEMINARY RD
8736026,75270,298948,1.0,GRIFFIN HOSPITAL,GRIFFIN HOSPITAL,130 DIVISION ST,130 DIVISION ST
8592336,40941,756787,1.0,DOCTORS HOSPITAL OF LAREDO,DOCTORS HOSPITAL OF LAREDO,10700 MCPHERSON ROAD,10700 MCPHERSON ROAD


In [8]:
joinDistinctTable = joinTable[joinTable['Facility Name'] != joinTable['Provider Name']]
joinDistinctTable.head(50)

,Account_Num,Provider_Num,Similar Score,Facility Name,Provider Name,Address,Provider Street Address
8800711,38285,515641,1.000000,ST MARYS MEDICAL CENTER,ST MARYS MEDICAL CENTER,2635 N 7TH ST,2635 N 7TH STREET
2149418,50717,347401,1.000000,MAYO CLINIC HLTH SYSTEM FRANCISCAN MED CTR,MAYO CLINIC HLTH SYSTEM FRANCISCAN MED CTR,700 WEST AVENUE SOUTH,700 WEST AVENUE SOUTH
10220592,52093,778470,1.000000,CONEY ISLAND HOSPITAL CENTER,CONEY ISLAND HOSPITAL,2601 OCEAN PARKWAY,2601 OCEAN PARKWAY
3263572,77076,449608,1.000000,NORTHRIDGE MEDICAL CENTER,NORTHRIDGE HOSPITAL MEDICAL CENTER,70 MEDICAL CENTER DRIVE,18300 ROSCOE BLVD
4510029,56087,824411,1.000000,NORTHRIDGE HOSPITAL MEDICAL CENTER,NORTHRIDGE MEDICAL CENTER,18300 ROSCOE BLVD,70 MEDICAL CENTER DRIVE
13079752,71151,266378,0.999999,ELLWOOD CITY MEDICAL CENTER,ELLWOOD CITY HOSPITAL,724 PERSHING STREET,724 PERSHING STREET
8583334,91370,410425,0.999998,MOUNTAINVIEW MEDICAL CENTER,MOUNTAINVIEW HOSPITAL,16 W MAIN ST,3100 N TENAYA WAY
15936,48460,847245,0.999998,INTERMOUNTAIN HOSPITAL,INTERMOUNTAIN MEDICAL CENTER,303 NORTH ALLUMBAUGH STREET,5121 SOUTH COTTONWOOD STREET
5274827,83610,265314,0.999998,REGIONAL HEALTH RAPID CITY HOSPITAL,RAPID CITY REGIONAL HOSPITAL,353 FAIRMONT BLVD POST OFFICE BOX 6000,353 FAIRMONT BLVD POST OFFICE BOX 6000
110287,73487,751124,0.999998,MEMORIAL HERMANN NORTHEAST HOSPITAL,MEMORIAL HERMANN NORTHEAST,18951 MEMORIAL NORTH,18951 MEMORIAL NORTH


In [9]:
#joinDistinctTable.to_csv('temp.csv')

In [ ]:
similarStrings = joinDistinctTable[joinDistinctTable['Similar Score'] > 0.998]
similarStrings.to_csv('softTF-IDF.csv')